In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"


In [2]:
from datasets import load_dataset

dataset = load_dataset("Spierocho/food_json_extract")


In [3]:
split_dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]

dataset = {"train": train_dataset, "validation": val_dataset}


In [4]:
from transformers import AutoTokenizer

model_name = "Qwen/Qwen3-0.6B-Base"
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [5]:
def format_prompt(sample):
    user = sample["input"]
    items = sample["output"]
    result = f"USER:{user}\nLIST_LENGTH:{len(items)}\n"
    result += "\n".join(
        [
            f"[{i}]FOOD:{item['food']},UNIT:{item['unit']},QUANTITY:{int(item['quantity'] or 0)}"
            for i, item in enumerate(items, 1)
        ]
    )
    result += tokenizer.eos_token
    return result

In [6]:
dataset["train"] = dataset["train"].map(lambda x: {"text": format_prompt(x)})
dataset["validation"] = dataset["validation"].map(lambda x: {"text": format_prompt(x)})

Map:   0%|          | 0/656 [00:00<?, ? examples/s]

Map:   0%|          | 0/165 [00:00<?, ? examples/s]

In [7]:
print(dataset["train"][0]["text"])


USER:I had 100g of Swedish meatballs, 150g of mashed potatoes, 50g lingonberry jam and a glass of milk.
LIST_LENGTH:4
[1]FOOD:Swedish meatballs,UNIT:gram,QUANTITY:100
[2]FOOD:mashed potatoes,UNIT:gram,QUANTITY:150
[3]FOOD:lingonberry jam,UNIT:gram,QUANTITY:50
[4]FOOD:milk,UNIT:glass,QUANTITY:1<|endoftext|>


In [8]:
print(tokenizer.tokenize(dataset["train"][0]["text"]))

['USER', ':I', 'Ġhad', 'Ġ', '1', '0', '0', 'g', 'Ġof', 'ĠSwedish', 'Ġmeat', 'balls', ',', 'Ġ', '1', '5', '0', 'g', 'Ġof', 'Ġmashed', 'Ġpotatoes', ',', 'Ġ', '5', '0', 'g', 'Ġling', 'on', 'berry', 'Ġjam', 'Ġand', 'Ġa', 'Ġglass', 'Ġof', 'Ġmilk', '.Ċ', 'LIST', '_LENGTH', ':', '4', 'Ċ', '[', '1', ']', 'FO', 'OD', ':', 'Sw', 'edish', 'Ġmeat', 'balls', ',', 'UNIT', ':', 'gram', ',', 'QU', 'ANTITY', ':', '1', '0', '0', 'Ċ', '[', '2', ']', 'FO', 'OD', ':m', 'ashed', 'Ġpotatoes', ',', 'UNIT', ':', 'gram', ',', 'QU', 'ANTITY', ':', '1', '5', '0', 'Ċ', '[', '3', ']', 'FO', 'OD', ':', 'ling', 'on', 'berry', 'Ġjam', ',', 'UNIT', ':', 'gram', ',', 'QU', 'ANTITY', ':', '5', '0', 'Ċ', '[', '4', ']', 'FO', 'OD', ':m', 'ilk', ',', 'UNIT', ':', 'glass', ',', 'QU', 'ANTITY', ':', '1', '<|endoftext|>']


In [9]:
max([len(tokenizer(sample["text"])["input_ids"]) for sample in dataset["train"]])


202

In [10]:
max([len(tokenizer(sample["text"])["input_ids"]) for sample in dataset["validation"]])


194

In [11]:
tokenizer.padding_side


'right'

In [12]:
def tokenize(sample):
    prompt = f"USER:{sample['input']}\n"
    prompt_len = len(tokenizer(prompt)["input_ids"])
    tokenized = tokenizer(
        sample["text"],
        padding="max_length",
        max_length=202,
        truncation=True,
    )
    pad_len = tokenized["input_ids"].count(tokenizer.pad_token_id)
    tokenized["labels"] = tokenized["input_ids"].copy()
    tokenized["labels"][:prompt_len] = [-100] * prompt_len
    if pad_len > 0:
        tokenized["labels"][-pad_len:] = [-100] * pad_len

    return tokenized


dataset["train"] = dataset["train"].map(tokenize, batched=False)
dataset["validation"] = dataset["validation"].map(tokenize, batched=False)


Map:   0%|          | 0/656 [00:00<?, ? examples/s]

Map:   0%|          | 0/165 [00:00<?, ? examples/s]

In [13]:
from transformers import (
    AutoModelForCausalLM,
    TrainingArguments,
)
from peft import LoraConfig
from trl import SFTTrainer


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    use_cache=False,
    attn_implementation="eager",
)


In [14]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
)


In [15]:
training_args = TrainingArguments(
    output_dir="./qwen-nutrition-lora",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=25,
    save_strategy="steps",
    save_steps=25,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to=[],
    weight_decay=0.01,
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    seed=887,
    fp16=True,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=peft_config,
    args=training_args,
)

trainer.train()



WARN  Python GIL is enabled: Multi-gpu quant acceleration for MoE models is sub-optimal and multi-core accelerated cpu packing is also disabled. We recommend Python >= 3.13.3t with Pytorch > 2.8 for mult-gpu quantization and multi-cpu packing with env `PYTHON_GIL=0`.
WARN  Feature `utils/Perplexity` requires python GIL or Python >= 3.13.3T (T for Threading-Free edition of Python) plus Torch 2.8. Feature is currently skipped/disabled.
INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.          


Truncating train dataset:   0%|          | 0/656 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/165 [00:00<?, ? examples/s]

INFO:root:cc -pthread -fno-strict-overflow -Wsign-compare -Wunreachable-code -DNDEBUG -g -O3 -Wall -fPIC -fPIC -c /tmp/tmp80x4xmgp/test.c -o /tmp/tmp80x4xmgp/test.o
INFO:root:cc -pthread /tmp/tmp80x4xmgp/test.o -laio -o /tmp/tmp80x4xmgp/a.out
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
INFO:root:cc -pthread -fno-strict-overflow -Wsign-compare -Wunreachable-code -DNDEBUG -g -O3 -Wall -fPIC -fPIC -c /tmp/tmpg7obdk8l/test.c -o /tmp/tmpg7obdk8l/test.o
INFO:root:cc -pthread /tmp/tmpg7obdk8l/test.o -L/usr/local/cuda-12.6 -L/usr/local/cuda-12.6/lib64 -lcufile -o /tmp/tmpg7obdk8l/a.out
INFO:root:cc -pthread -fno-strict-overflow -Wsign-compare -Wunreachable-code -DNDEBUG -g -O3 -Wall -fPIC -fPIC -c /tmp/tmpejqiia1m/test.c -o /tmp/tmpejqiia1m/test.o
INFO:root:cc -pthread /tmp/tmpejqiia1m/test.o -laio -o /tmp/tmpejqiia1m/a.out
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
The tokenizer 

Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
25,0.028500,0.018737,2.346663,158368.000000,0.995557
50,0.006000,0.015011,2.032435,316736.000000,0.996296


TrainOutput(global_step=63, training_loss=0.10414779473037, metrics={'train_runtime': 124.9194, 'train_samples_per_second': 15.754, 'train_steps_per_second': 0.504, 'total_flos': 1074682920960000.0, 'train_loss': 0.10414779473037, 'epoch': 3.0})

In [16]:
trainer.save_model("./qwen-nutrition-lora")


In [17]:
from transformers import logging

logging.set_verbosity_error()


In [18]:
from transformers import pipeline
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map="cuda")
model = PeftModel.from_pretrained(base_model, "./qwen-nutrition-lora", device_map="cuda")
text_gen = pipeline("text-generation", model=model, tokenizer=tokenizer)


In [19]:
from tqdm import tqdm


def evaluate_accuracy(
    dataset,
    log_file,
    batch_size=32,
):
    correct = 0
    total = len(dataset)

    with open(log_file, "w") as file:
        for i in tqdm(range(0, total, batch_size)):
            texts = [
                text.replace(tokenizer.eos_token, "")
                for text in dataset[i : i + batch_size]["text"]
            ]
            prefixes = [text.split("\n")[0] + "\n" for text in texts]
            print(repr(prefixes[0]))
            gen_outs = text_gen(
                prefixes,
                num_beams=1,
                do_sample=False,
                batch_size=batch_size,
            )
            for text, gen_out in zip(texts, gen_outs):
                gen_text = gen_out[0]["generated_text"]
                if len(gen_text) >= len(text) and text == gen_text[: len(text)]:
                    correct += 1
                else:
                    print(f"Mismatch:\n{text} ->\n{gen_text}\n", file=file)

    return correct / total


In [20]:
acc = evaluate_accuracy(dataset["validation"], log_file="eval.log")
print(f"Validation accuracy: {acc:.4f}")


  0%|                                                                                                                                                       | 0/6 [00:00<?, ?it/s]

'USER:I had a slice of quiche with 100g of mixed greens and a serving of 200ml of iced tea.\n'


 17%|███████████████████████▊                                                                                                                       | 1/6 [00:14<01:13, 14.66s/it]

'USER:Consumed 150g of quinoa, 100g of grilled chicken breast, a side of steamed broccoli, and a glass of protein shake.\n'


 33%|███████████████████████████████████████████████▋                                                                                               | 2/6 [00:29<00:59, 14.78s/it]

...

'USER:Lunch: 300g  Pancit (Filipino), 200g  Kastike (Finnish), 180g  Samosa Chaat (Indian), 150g  Frijoles Negros (Cuban), 250ml  Calamansi Juice (Filipino).\n'


 83%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏                       | 5/6 [01:14<00:14, 14.84s/it]

'USER:Had 150g of crab meat, a pineapple, 50g of chia seeds, and 100g of bok choy.\n'


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:27<00:00, 14.54s/it]

Validation accuracy: 0.2061


In [21]:
%cat eval.log


Mismatch:
USER:I had a slice of quiche with 100g of mixed greens and a serving of 200ml of iced tea.
LIST_LENGTH:3
[1]FOOD:quiche,UNIT:slice,QUANTITY:1
[2]FOOD:mixed greens,UNIT:gram,QUANTITY:100
[3]FOOD:iced tea,UNIT:milliliter,QUANTITY:200 ->
USER:I had a slice of quiche with 100g of mixed greens and a serving of 200ml of iced tea.
:FOOD:quiche,UNIT:slice,QUANTITY:1
:FOOD:mixed greens,UNIT:gram,QUANTITY:100
:FOOD:iced tea,UNIT:milliliter,QUANTITY:200
LIST_LENGTH:3
[1]FOOD:quiche,UNIT:slice,QUANTITY:1
[2]FOOD:mixed greens,UNIT:gram,QUANTITY:100
[3]FOOD:iced tea,UNIT:milliliter,QUANTITY:200

Mismatch:
USER:Norwegian: 160g of  smalahove, 100g of  brunost, 140g of  fiskesuppe.
LIST_LENGTH:3
[1]FOOD:smalahove,UNIT:gram,QUANTITY:160
[2]FOOD:brunost,UNIT:gram,QUANTITY:100
[3]FOOD:fiskesuppe,UNIT:gram,QUANTITY:140 ->
USER:Norwegian: 160g of  smalahove, 100g of  brunost, 140g of  fiskesuppe.
FOOD:smahalahove,UNIT:gram,QUANTITY:160
FOOD:brunost,UNIT:gram,QUANTITY:100
FOOD:fiskesuppe,UNIT:gram,